In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt; plt.rc('font', size=12)
import seaborn as sns
from scipy.stats import entropy
import re

In [2]:
def norm(x):
    x = x.copy()
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            if np.isnan(x[i,j]):
                continue
            x[i,j] = (x[i,j] - x_min) / (x_max - x_min)
    x = x
    return x

def softmax(x):
    x = x.copy()
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            if np.isnan(x[i,j]):
                x[i,j] = 0
            elif x[i,j] == 0:
                x[i,j] = 0
            else:
                x[i,j] = np.exp(x[i,j])
    x = x / np.sum(x)
    return x

def softmin(x):
    x = x.copy()
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            if np.isnan(x[i,j]):
                x[i,j] = 0
            elif x[i,j] == 0:
                x[i,j] = 0
            else:
                x[i,j] = np.exp(-x[i,j])
    x = x / np.sum(x)
    return x

In [3]:
eco_lf0 = pd.read_table('ecological-modeling/mcnulty-results/mcnulty-LF0-seed96/posteriors/interactions/mean_matrix_transf.tsv', index_col=0)
eco_hf0 = pd.read_table('ecological-modeling/mcnulty-results/mcnulty-HF0-seed12/posteriors/interactions/mean_matrix_transf.tsv', index_col=0)

In [21]:
metabo_carbo = pd.read_table('metabolic-modeling/metabolic-metrics/metaddinfo_carbo.tsv', index_col=0)
metabo_amino = pd.read_table('metabolic-modeling/metabolic-metrics/metaddinfo_amino.tsv', index_col=0)
metabo_carbo_amino = pd.read_table('metabolic-modeling/metabolic-metrics/metaddinfo_carbo_amino.tsv', index_col=0)
metabo_int = pd.read_table('metabolic-modeling/metabolic-metrics/metaddinfo_ecs_int.tsv', index_col=0)

In [12]:
matrix1 = softmax(eco_lf0.values)
matrix2 = metabo_carbo.values/metabo_carbo.values.sum()

In [13]:
matrix1

array([[0.        , 0.        , 0.00946971, 0.00954   , 0.        ,
        0.        , 0.00939873, 0.00960442, 0.00935801, 0.00957114,
        0.00944721],
       [0.        , 0.        , 0.00943774, 0.00948399, 0.01189121,
        0.01170362, 0.00931945, 0.01169358, 0.00934644, 0.00953896,
        0.00939709],
       [0.01174408, 0.0117909 , 0.        , 0.0096015 , 0.0119094 ,
        0.01173912, 0.00935735, 0.00959649, 0.00940886, 0.00952462,
        0.00948407],
       [0.        , 0.        , 0.00959973, 0.        , 0.01168062,
        0.        , 0.00944504, 0.01181549, 0.00944817, 0.00957407,
        0.00954484],
       [0.        , 0.0116835 , 0.00959049, 0.00961549, 0.        ,
        0.01168789, 0.00941039, 0.01182725, 0.00945689, 0.00956098,
        0.0095112 ],
       [0.        , 0.01171181, 0.00963029, 0.00961591, 0.01181787,
        0.        , 0.00943407, 0.01178458, 0.0094009 , 0.00964986,
        0.009607  ],
       [0.00961384, 0.00965902, 0.00959145, 0.00955506, 0.

In [16]:
matrix2

array([[0.        , 0.01776371, 0.01387999, 0.0138098 , 0.0109049 ,
        0.00989032, 0.00126274, 0.00295343, 0.00364784, 0.00631522,
        0.00351664],
       [0.00316931, 0.        , 0.00246572, 0.00281437, 0.00167869,
        0.00218615, 0.00083872, 0.00295343, 0.00394591, 0.00362784,
        0.00351664],
       [0.00138068, 0.00456081, 0.        , 0.00246854, 0.0038096 ,
        0.00087847, 0.00083872, 0.00252941, 0.00407186, 0.00236792,
        0.00351664],
       [0.00229477, 0.00589374, 0.00345282, 0.        , 0.00517815,
        0.00179256, 0.00083872, 0.00252941, 0.00317606, 0.00329932,
        0.00351664],
       [0.00298021, 0.00834839, 0.00838422, 0.00876848, 0.        ,
        0.0067544 , 0.00083872, 0.00295343, 0.00381892, 0.00351299,
        0.00301864],
       [0.00399603, 0.01088625, 0.00748349, 0.0074133 , 0.0087848 ,
        0.        , 0.00083872, 0.00295343, 0.0049199 , 0.00461914,
        0.00351664],
       [0.01518201, 0.02935239, 0.0272573 , 0.02627302, 0.

In [17]:
eco = [eco_lf0, eco_hf0]
eco_name = ['LF0', 'HF0']
metabo = [metabo_carbo, metabo_amino]
metabo_name = ['carbohydrate', 'amino acid']
for i in range(len(eco)):
    for j in range(len(metabo)):
        matrix1 = softmax(eco[i].values)
        matrix2 = metabo[j].values/metabo[j].values.sum()
        mean_matrix = (matrix1 + matrix2) / 2
        js_div = (entropy(matrix1.flatten(), mean_matrix.flatten()) + entropy(matrix2.flatten(), mean_matrix.flatten())) / 2
        print(f'{eco_name[i]} ecological interactions with {metabo_name[j]} metabolism js divergence: {js_div}')

LF0 ecological interactions with carbohydrate metabolism js divergence: 0.1266268993041118
LF0 ecological interactions with amino acid metabolism js divergence: 0.09200746345395415
HF0 ecological interactions with carbohydrate metabolism js divergence: 0.22595808897158037
HF0 ecological interactions with amino acid metabolism js divergence: 0.15818782243975527


In [19]:
for i in range(len(eco)):
    matrix1 = softmax(eco[i].values)
    matrix2 = metabo_carbo_amino.values/metabo_carbo_amino.values.sum()
    mean_matrix = (matrix1 + matrix2) / 2
    js_div = (entropy(matrix1.flatten(), mean_matrix.flatten()) + entropy(matrix2.flatten(), mean_matrix.flatten())) / 2
    print(f'{eco_name[i]} ecological interactions with carbohydrate and amino acid metabolism js divergence: {js_div}')

LF0 ecological interactions with carbohydrate and amino acid metabolism js divergence: 0.09117987906608682
HF0 ecological interactions with carbohydrate and amino acid metabolism js divergence: 0.1785906549315783


In [22]:
matrix1 = softmax(eco_lf0.values)
matrix2 = metabo_int.values/metabo_int.values.sum()
mean_matrix = (matrix1 + matrix2) / 2
js_div = (entropy(matrix1.flatten(), mean_matrix.flatten()) + entropy(matrix2.flatten(), mean_matrix.flatten())) / 2
print(f'{eco_name[i]} ecological interactions with carbohydrate and amino acid metabolism js divergence: {js_div}')

HF0 ecological interactions with carbohydrate and amino acid metabolism js divergence: 0.0761122000657101


In [23]:
matrix1 = softmax(eco_hf0.values)
matrix2 = metabo_int.values/metabo_int.values.sum()
mean_matrix = (matrix1 + matrix2) / 2
js_div = (entropy(matrix1.flatten(), mean_matrix.flatten()) + entropy(matrix2.flatten(), mean_matrix.flatten())) / 2
print(f'{eco_name[i]} ecological interactions with carbohydrate and amino acid metabolism js divergence: {js_div}')

HF0 ecological interactions with carbohydrate and amino acid metabolism js divergence: 0.15484177224652054
